# Bode plot

## List connected devices

In [ ]:
import JDS6600
import SDS1202XE
from bodeplot import bodeplot
import time
import math

rm = SDS1202XE.get_resource_manager()
SDS1202XE.list_resources(rm)
JDS6600.list_com_ports()

## Connect devices

In [ ]:
gen = JDS6600.connect("COM5")
sds = SDS1202XE.connect(rm, "USB0::0xF4ED::0xEE3A::SDS1EDEQ4R7911::INSTR")



## Initialize

In [ ]:
MAXV = 2

JDS6600.set_waveform(gen, 1, JDS6600.SINE)
JDS6600.set_frequency(gen, 1, 10)
JDS6600.set_amplitude(gen, 1, MAXV)
JDS6600.set_offset(gen, 1, 0)
JDS6600.set_output(gen, channel1=1, channel2=0)

time.sleep(1)
#SDS1202XE.autosetup(sds)
#time.sleep(3)
SDS1202XE.set_vertical(sds, channel=1, volt_by_div = "1V")
SDS1202XE.set_vertical(sds, channel=2, volt_by_div = "500mV")
SDS1202XE.set_tdiv(sds, tdiv="20MS")
SDS1202XE.set_measure(sds, channel=1)
SDS1202XE.set_phase_measure(sds)
time.sleep(1)

## Launch measures

In [ ]:
frequencies = [10, 20, 50, 100, 200, 500, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 18000, 20000, 22000]

gains = []
phases = []

from tqdm import tqdm

for i in tqdm(range(len(frequencies))):
    f = frequencies[i]
    JDS6600.set_frequency(gen, 1, f)
    time.sleep(1.0)
    v = SDS1202XE.get_measure_vpp(sds, channel=1)
    p = SDS1202XE.get_phase(sds)
    gains.append(10*math.log(v/MAXV, 10))
    phases.append(p)


## Bode plot

In [ ]:
bodeplot(frequencies, gains, phases)
